In [1]:
import copy
import numpy as np
import torch
from tqdm import tqdm
from sklearn.utils import check_random_state
import pandas as pd
from proplace.clfutils import HiddenPrints
import proplace
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


D:\Apps\anaconda\envs\proplace\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [2]:
# load dataset utils
dataset = proplace.InnDataSet("adult")

In [5]:
# get a clf#
torch.manual_seed(5)
np.random.seed(1)
random_state = check_random_state(10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
d = dataset
torch_model = proplace.train_clf(d.X1_train, d.y1_train, d.X1_test, d.y1_test, 10, epochs=20, data_name="adult", save_clf=False,
                        load_clf=False)
model = proplace.InnModel(dataset, torch_model, 10)

Using cuda device
Epoch 0: train loss: 0.6619033813476562
Epoch 0: train loss: 0.6377777457237244
Epoch 0: train loss: 0.6543306708335876
Epoch 0: train loss: 0.5638924837112427
Epoch 0: train loss: 0.5632845163345337
Epoch 0: train loss: 0.6056914925575256
Epoch 0: train loss: 0.4965665638446808
Epoch 0: train loss: 0.5215808153152466
Epoch 0: train loss: 0.47086775302886963
Epoch 0: train loss: 0.582423985004425
Epoch 0: train loss: 0.6502950191497803
Epoch 0: train loss: 0.3909924030303955
Epoch 0: train loss: 0.5300584435462952
Epoch 0: train loss: 0.41422539949417114
Epoch 0: train loss: 0.38719794154167175
Epoch 0: train loss: 0.547995388507843
Epoch 0: train loss: 0.6657342910766602
Epoch 0: train loss: 0.4612656831741333
Epoch 0: train loss: 0.39031460881233215
Epoch 0: train loss: 0.19671513140201569
Epoch 0: train loss: 0.5305477976799011
Epoch 0: train loss: 0.3901253640651703
Epoch 0: train loss: 0.35245856642723083
Epoch 0: train loss: 0.37812596559524536
Epoch 0: train lo

In [6]:
m2s = proplace.retrain_models(dataset, 10, 20) + proplace.retrain_models_leave_some_out(dataset, 10, 20)

100%|██████████| 10/10 [01:40<00:00, 10.09s/it]


## Prepare for exps

In [7]:
from exputils import *
clf = model
test_xs_vals, test_xs, test_xs_carla, utildataset, nodes = get_test_data(model, m2s, dataset)

## Run Baselines

In [8]:
# ours
rnnce_ces, treer, X_class1_clf_robust = run_rnnce(d, clf, nodes, utildataset, 0.035, test_xs_vals)
proplace_ces = run_proplace(clf, nodes, utildataset, 0.035, treer, X_class1_clf_robust, test_xs_vals)
proplace_scores = eval_ces(clf, m2s, proplace_ces, d, utildataset, test_xs_vals, 0.035)

0it [00:00, ?it/s]

Set parameter Username
[INFO] Set parameter Username [inn.py __init__]
Academic license - for non-commercial use only - expires 2024-01-02
[INFO] Academic license - for non-commercial use only - expires 2024-01-02 [inn.py __init__]


3984it [00:43, 91.60it/s]
50it [00:04, 10.20it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:10,  4.81it/s]

1.0 0.04174017624277112 >>> robustness - delta validity, avg bound
0.04618485279565781 >>> l1 cost
0.84 >>> percentage of inliers dataset class 1 points
1.224351494591788 >>> avg lof dataset class 1 points


In [29]:
wach_ces = run_wachter(clf, test_xs_carla)
wach_scores = eval_ces(clf, m2s, wach_ces, d, utildataset, test_xs_vals,  delta=0.035)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

50it [00:08,  5.67it/s]

0.782608695652174 0.3771112915897843 >>> robustness - delta validity, avg bound
0.17526848468829617 >>> l1 cost
0.45652173913043476 >>> percentage of inliers dataset class 1 points
1.5867056355065423 >>> avg lof dataset class 1 points


In [30]:
rbr_ces = run_rbr(clf, d, random_state, test_xs_vals)
rbr_scores = eval_ces(clf, m2s, rbr_ces, d, utildataset, test_xs_vals, 0.035)

50it [06:51,  8.23s/it]


1.0 >>> validity
0.999 >>> robustness - M2 validity


50it [00:09,  5.20it/s]

0.0 -1.3007599020130527 >>> robustness - delta validity, avg bound
0.030798483089267407 >>> l1 cost
0.78 >>> percentage of inliers dataset class 1 points
1.2802430629184132 >>> avg lof dataset class 1 points


In [31]:
robx_ces = run_robx(clf, d, test_xs_vals)
robx_scores = eval_ces(clf, m2s, robx_ces, d, utildataset, test_xs_vals, 0.035)

3984it [00:03, 1268.08it/s]


(644, 13)


50it [00:00, 88.07it/s] 


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:09,  5.08it/s]

0.82 0.29318066668975445 >>> robustness - delta validity, avg bound
0.06372724414355009 >>> l1 cost
0.94 >>> percentage of inliers dataset class 1 points
1.2824211810105532 >>> avg lof dataset class 1 points


In [28]:
roar_params = {
        "lr": 0.04,
        "lambda_": 0.1,
        "delta_max": 0.001,
        "norm": 1,
        "t_max_min": 0.5,
        "loss_type": "MSE",
        "y_target": [1],
        "binary_cat_features": False,
        "loss_threshold": 1e-3,
        "discretize": False,
        "sample": False,
        "lime_seed": 0,
        "seed": 0,
    }
roar_ces = run_roar(clf, test_xs_carla, roar_params)
roar_scores = eval_ces(clf, m2s, roar_ces, d, utildataset, test_xs_vals, delta=0.035)

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
1.0 >>> validity
0.9930000000000001 >>> robustness - M2 validity


50it [00:10,  4.87it/s]

0.98 15.167361491530633 >>> robustness - delta validity, avg bound
0.27851425672144037 >>> l1 cost
0.3 >>> percentage of inliers dataset class 1 points
1.9574978907080236 >>> avg lof dataset class 1 points


In [32]:
protor_ces = run_proto_r(clf, d, utildataset, test_xs_vals, nodes, delta_target=0.035)
protor_scores = eval_ces(clf, m2s, protor_ces, d, utildataset, test_xs_vals, 0.035)

0it [00:00, ?it/s]

[WARNING] From D:\Apps\anaconda\envs\proplace\lib\site-packages\alibi\explainers\cfproto.py:123: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.
 [module_wrapper.py _tfmw_add_deprecation_warning]
[WARNING] From D:\Apps\anaconda\envs\proplace\lib\site-packages\tensorflow_core\python\ops\array_ops.py:1475: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where [deprecation.py new_func]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No counterfactual found! [cfproto.py explain]


1it [01:17, 77.77s/it]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


2it [02:11, 63.55s/it]

...

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No counterfactual found! [cfproto.py explain]


49it [1:03:57, 106.30s/it]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


50it [1:05:10, 78.21s/it] 

ValueError: setting an array element with a sequence.

In [47]:
protor_ces_2 = np.zeros((50, 13), dtype=np.float32)
for i, item in enumerate(protor_ces):
    protor_ces_2[i] = protor_ces[i]

In [49]:
protor_scores = eval_ces(clf, m2s, np.array(protor_ces_2), d, utildataset, test_xs_vals, 0.035)

0.98 >>> validity
0.9760000000000003 >>> robustness - M2 validity


50it [00:10,  4.83it/s]

0.5510204081632653 0.22769164850432436 >>> robustness - delta validity, avg bound
0.0683732651316049 >>> l1 cost
0.46938775510204084 >>> percentage of inliers dataset class 1 points
1.5972621198684944 >>> avg lof dataset class 1 points


In [13]:
milpr_ces = run_milpr(clf, d, nodes, utildataset, 0.05, test_xs_vals)
milpr_scores = eval_ces(clf, m2s, milpr_ces, d, utildataset, test_xs_vals, 0.035)

50it [00:26,  1.89it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:10,  4.74it/s]

1.0 1.2978658000326657 >>> robustness - delta validity, avg bound
0.024252494897947813 >>> l1 cost
0.4 >>> percentage of inliers dataset class 1 points
1.689919459640948 >>> avg lof dataset class 1 points


In [50]:
# make table
from tabulate import tabulate
scores_names = ["name", "validity", "delta validity", "M2 validity", "l1", "%inlier class 1", "lof class 1"]
scores_table = [scores_names,
                np.concatenate((['wach'], np.round(wach_scores, 3))),
                np.concatenate((['rbr'], np.round(rbr_scores, 3))),
                np.concatenate((['robx'], np.round(robx_scores, 3))),
                np.concatenate((['roar'], np.round(roar_scores, 3))),
                np.concatenate((['proto-r'], np.round(protor_scores, 3))),
                np.concatenate((['milp-r'], np.round(milpr_scores, 3))),
                np.concatenate((['proplace'], np.round(proplace_scores, 3))), ]
print(tabulate(scores_table, headers='firstrow', tablefmt='outline'))

+----------+------------+------------------+---------------+-------+-------------------+---------------+
| name     |   validity |   delta validity |   M2 validity |    l1 |   %inlier class 1 |   lof class 1 |
+==========+============+==================+===============+=======+===================+===============+
| wach     |       0.92 |            0.783 |         0.894 | 0.175 |             0.457 |         1.587 |
| rbr      |       1    |            0     |         0.999 | 0.031 |             0.78  |         1.28  |
| robx     |       1    |            0.82  |         1     | 0.064 |             0.94  |         1.282 |
| roar     |       1    |            0.98  |         0.993 | 0.279 |             0.3   |         1.957 |
| proto-r  |       0.98 |            0.551 |         0.976 | 0.068 |             0.469 |         1.597 |
| milp-r   |       1    |            1     |         1     | 0.024 |             0.4   |         1.69  |
| proplace |       1    |            1     |         1 